In [81]:
from selenium import webdriver
from bs4 import BeautifulSoup as soup
import re

page2 = 'https://www.opentable.com/s?covers=1&latitude=40.751637&longitude=-73.986409&corrid=bb294e9c-7c9b-48ae-997c-d1999436a6be&sortBy=rating&metroId=8&queryUnderstandingType=none&page=2'

In [34]:
# start web browser
driver=webdriver.Chrome()

# get source code
driver.get(page2)
y = 500
# scroll down page incrementally to load restaurant elements
for timer in range(0,50):
     driver.execute_script("window.scrollTo(0, "+str(y)+")")
     y += 500
     time.sleep(0.05)
results_page = driver.page_source
time.sleep(2)

# close web browser
driver.close()

In [273]:
results_html = soup(results_page, 'html.parser')
restaurants = results_html.find_all('div', attrs = {"class" : "_3uVfVbI1iLfMbszbU6KoOL"})
len(restaurants)

NameError: name 'results_page' is not defined

In [272]:
unloaded = results_html.find_all('div', attrs = {"class" : "_2XzcUvJpjP8jz_0tEdigq0"})
len(unloaded)

NameError: name 'results_html' is not defined

In [73]:
rest_0 = restaurants[0]

rest_0_a = rest_0.find('a', attrs = {"class":"_1e9PcCDb012hY4BcGfraQB"})
rest_0_a.get('href')

'https://www.opentable.com/r/coarse-nyc-new-york-2?corrid=9a2f5a53-4925-4f21-9380-46d641cb1616&avt=eyJ2IjoyLCJtIjoxLCJwIjoxLCJzIjowLCJuIjowfQ&p=1&sd=2021-07-17T19%3A00%3A00'

In [113]:
rest_list = []
for restaurant in restaurants:
    rest_dict = {}
    
    restaurant_child = restaurant.find('a', attrs = {"class":"_1e9PcCDb012hY4BcGfraQB"})
    # get restaurant name
    rest_dict['name'] = restaurant_child.get('aria-label')
    
    # get restaurant url
    url = restaurant_child.get('href')
    rest_dict['url'] = url
    
    # get promoted status
    rest_dict['promoted'] = 1 if restaurant.get('data-promoted') == 'true' else 0
    
    # get price tier
    rest_dict['price_tier'] = restaurant.find('span', attrs = {"class":"_3sSkv7iJ6Tl1VxRjYAjp13"}).get_text()
    
    # get times booked today
    times_booked_today = restaurant.find('span', attrs = {"class": "_2VIffaVUDxw_-tEh-6XOB_ _2EluNCOTdgGq9H4SxGZwUg"}).string
    if not times_booked_today is None:
        rest_dict['bookings_today'] = int(re.search('\d+', times_booked_today).group(0))
    else:
        rest_dict['bookings_today'] = 0
        
    rest_list.append(rest_dict)

In [224]:
a = rest_list[1]
a['url']

'https://www.opentable.com/r/merchants-cigar-bar-new-york?corrid=9a2f5a53-4925-4f21-9380-46d641cb1616&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=1&sd=2021-07-17T19%3A00%3A00'

In [225]:
rest_list[1]

{'url': 'https://www.opentable.com/r/merchants-cigar-bar-new-york?corrid=9a2f5a53-4925-4f21-9380-46d641cb1616&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=1&sd=2021-07-17T19%3A00%3A00',
 'name': 'Merchants Cigar Bar restaurant',
 'promoted': 0,
 'price_tier': '$$$$',
 'bookings_today': 5}

In [270]:
# def get_restaurant_info(url, curr_rest_dict):
subdriver=webdriver.Chrome()
subdriver.get(a['url'])
subdriver.maximize_window() # maximize to make sure sidebar is loaded
rest_html = subdriver.page_source
time.sleep(0.05)
subdriver.close()

curr_rest = soup(rest_html, 'html.parser')
curr_rest_dict = {}

# get number of reviews
curr_rest_dict['review_count'] = int(re.search('\d+', curr_rest.find('div', attrs = {"class" : "oc-reviews-c57b151d"}).string).group(0))

# get overall rating and subratings
curr_rest_dict['overall'] = float(curr_rest.find('div', attrs = {"class" : "oc-reviews-491257d8"}).span.string)
subreviews = curr_rest.find_all('div', attrs = {"class" : "oc-reviews-15d38b07"})
curr_rest_dict['food'] = float(subreviews[0].string)
curr_rest_dict['service'] = float(subreviews[1].string)
curr_rest_dict['ambience'] = float(subreviews[2].string)
curr_rest_dict['value'] = float(subreviews[3].string)

# get noise level
curr_rest_dict['noise'] = curr_rest.find('span', attrs = {"class" : "oc-reviews-624ebf8b"}).string

# get percent of reviewers who would recommended to a friend
rec_string = re.search('\d+%', curr_rest.find_all('div', attrs = {"class" : "oc-reviews-dfc07aec"})[1].get_text()).group(0)
curr_rest_dict['pct_recommended'] = int(re.search('\d+', rec_string).group(0))

# add placeholder keys for sidebar information
placeholder = ['neighborhood', 'hours', 'cuisines', 'dining_style', 'dress_code', 'chef', 'tags']
for i in range(len(placeholder)):
    curr_rest_dict[placeholder[i]] = None

# zip (field, value) tuples of restaurant page sidebar information into details_list
sidebar = curr_rest.find('div', attrs = {"class":"_1e466fbf"})
details = sidebar.find_all('div', attrs = {"class":"df8add00"})
details_list = [zip(item.find_all('div', attrs = {"class":"c3981cf8 _965a91d5"}), 
                    item.find_all('div', attrs = {"class":"e7ff71b6 b2f6d1a4"})) for item in details] 
for i in range(len(details_list)):
    for x, y in details_list[i]:
        details_list[i] = (x.string, y.string)

# filter details_list for desired information, add info to dict under placeholder keys
desired_details = zip(['Neighborhood', 'Hours of operation', 'Cuisines', 'Dining Style', 'Dress code', '(?i)(.*chef.*)', 'Additional'], placeholder)

for x, y in desired_details:
    for a, b in details_list:
        if re.search(x, a):
            curr_rest_dict[y] = b            

# add placeholder keys for safety information
safety_categories = ['Cleaning & Sanitizing', 'Physical Distancing', 'Protective Equipment','Screening']
for i in range(len(safety_categories)):
    curr_rest_dict[safety_categories[i]] = 0
    
# get COVID-19 safety information
safety_html = curr_rest.find_all('div', attrs = {"class" : "_77b505d0 _965a91d5"})
safety_features = [item.find('span').string for item in safety_html]

for i in range(len(safety_categories)):
    for j in range(len(safety_features)):
        if safety_categories[i]==safety_features[j]:
            curr_rest_dict[safety_categories[i]] = 1

curr_rest_dict

{'review_count': 24,
 'overall': 4.9,
 'food': 4.6,
 'service': 4.8,
 'ambience': 4.7,
 'value': 4.5,
 'noise': 'Quiet',
 'pct_recommended': 86,
 'neighborhood': 'Upper East Side',
 'hours': 'Wed–Sun 5:00 pm–11:00 pm',
 'cuisines': 'American',
 'dining_style': 'Casual Elegant',
 'dress_code': 'Smart Casual',
 'chef': 'Antemlo Ambrosio',
 'tags': 'Bar/Lounge, Beer, Cigar Room, Cocktails, Corkage Fee, Full Bar, Gluten-free Menu, Indoor Smoking Area, Wine',
 'Cleaning & Sanitizing': 1,
 'Physical Distancing': 1,
 'Protective Equipment': 1,
 'Screening': 1}

In [65]:
unloaded = text.find_all('div', attrs = {"class" : "_2XzcUvJpjP8jz_0tEdigq0"})
len(unloaded)

0